# Configurações

In [ ]:
!pip install transformers[sentencepiece]

In [ ]:
!pip install datasets

# Import's

In [1]:
from transformers import pipeline
from transformers import AutoTokenizer
from datasets import load_dataset
import os

/home/thiago/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-07-16 12:08:33.665373: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-16 12:08:33.665466: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Código

In [2]:
DIR = './'
dataset_path = "Dataset/"
remove_dataset = 'concatenated.csv' 
file_dataset_facebook = 'facebook/'
file_dataset_twitter = 'twitter/'
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

## Baixando o Modelo Treinado

In [ ]:
from transformers import TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(model_path)
model.save_pretrained(model_path)

2022-07-16 12:08:39.711230: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-07-16 12:08:39.711638: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-07-16 12:08:39.711700: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Thiaguinho-PC): /proc/driver/nvidia/version does not exist
2022-07-16 12:08:39.717464: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-16 12:08:40.827626: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 768006144 exc

In [ ]:
pipeline_as = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)

## Lendo os datasets

### Dataset's "Brazilian Portuguese Sentiment Analysis Datasets" 

In [ ]:
## Pegando os dataset's "Brazilian Portuguese Sentiment Analysis Datasets" (https://www.kaggle.com/datasets/fredericods/ptbr-sentiment-analysis-datasets?resource=download)
list_file = [file for file in os.listdir(DIR) if file.endswith('.csv')] ## pegando os arquivos ".csv"
list_file.remove(dataset_concatenated)

##### Erro
Erro: Quando passamos uma lista de datasets para serem carregados, ocorre um erro, porém se carregar um por um esse erro não ocorre, e se carregar usando o dataset concatenado o erro acontece.
Visto que o dataset do "buscape.csv", em sua coluna "original_index" o tipo é uma "string" enquando nos demais dataset's essa mesma coluna é "int64", causando um conflito no merge realizado pela a biblioteca dataset

In [ ]:
## dataset_BPSA = load_dataset("csv", data_files = list_file)

In [ ]:
## Metodo funcional
dataset_BPSA = []
for file in list_file:
    dataset_BPSA.append(load_dataset("csv", data_files = file))
    dataset_BPSA[-1]['dataset_name'] = file

In [ ]:
#print(dataset_BPSA)

#### Analisando

In [ ]:
print("Quantidade de informações: " + str(sum([len(x['train']) for x in dataset_BPSA])))

In [ ]:
dataset_BPSA[0]['train'][142]

In [ ]:
for i in range(len(dataset_BPSA)):
    print(dataset_BPSA[i]['train'].features)
    print("------------------------------------------------")

##### Avaliando tamanho dos textos

In [ ]:
for dataset in dataset_BPSA:
    cont = 0
    for data in dataset['train']:
        resultado = 0
        if data['review_text_processed'] is not None:
            resultado = len(data['review_text_processed'].split(' '))
        if resultado >= 512:
            cont += 1
    print(str(dataset['dataset_name']) + " - " + str(cont))

##### Teste manual com pipeline

In [ ]:
print("Texto: " + str(dataset_BPSA[0]['train'][0]['review_text']))
print(pipeline_as(dataset_BPSA[0]['train'][0]['review_text']))

In [ ]:
print("Texto: " + str(dataset_BPSA[0]['train'][9]['review_text_processed']))
print(pipeline_as(dataset_BPSA[0]['train'][9]['review_text_processed']))

In [ ]:
print("Texto: " + str(dataset_BPSA[0]['train'][2]['review_text_processed']))
print(pipeline_as(dataset_BPSA[0]['train'][2]['review_text_processed']))

#### Dataset's (Facebook comentários de página de cachaça)

In [ ]:
## Pegando os dataset's de comentários do facebook de páginas de cachaças (extração realizada pelo autor via API)
list_file = [DIR + file_dataset_facebook + file for file in os.listdir(DIR + file_dataset_facebook) if file.endswith('.csv')] ## pegando os arquivos ".csv"

In [ ]:
dataset_facebook = load_dataset("csv", data_files = list_file)

#### Analisando

In [ ]:
print("Quantidade de informações: " + str(len(dataset_facebook['train'])))

In [ ]:
dataset_facebook['train'][0]

In [ ]:
dataset_facebook['train'].features

##### Teste manual com pipeline

In [ ]:
print("Texto: " + str(dataset_facebook['train'][55]['message']))
print(pipeline_as(dataset_facebook['train'][55]['message']))

In [ ]:
print("Texto: " + str(dataset_facebook['train'][102]['message']))
print(pipeline_as(dataset_facebook['train'][102]['message']))

In [ ]:
print("Texto: " + str(dataset_facebook['train'][86]['message']))
print(pipeline_as(dataset_facebook['train'][86]['message']))

#### Dataset's (Twitter comentários de página de cachaça)

In [ ]:
## Pegando os dataset's de comentários do twitter de páginas de cachaças (extração realizada pelo autor via API)
list_file = [DIR + file_dataset_twitter + file for file in os.listdir(DIR + file_dataset_twitter) if file.endswith('.csv')] ## pegando os arquivos ".csv"

In [ ]:
dataset_twitter = load_dataset("csv", data_files = list_file)

#### Analisando

In [ ]:
print("Quantidade de informações: " + str(len(dataset_twitter['train'])))

In [ ]:
dataset_twitter['train'][0]

In [ ]:
dataset_twitter['train'].features

##### Teste manual com pipeline

In [ ]:
i = 55
print("Texto: " + str(dataset_twitter['train'][i]['texto']) + "\nResultado (TextBlob): " + str(dataset_twitter['train'][i]['score'])) 
print(pipeline_as(dataset_twitter['train'][i]['texto']))

In [ ]:
i = 1000
print("Texto: " + str(dataset_twitter['train'][i]['texto']) + "\nResultado (TextBlob): " + str(dataset_twitter['train'][i]['score'])) 
print(pipeline_as(dataset_twitter['train'][i]['texto']))

In [ ]:
i = 99
print("Texto: " + str(dataset_twitter['train'][i]['texto']) + "\nResultado (TextBlob): " + str(dataset_twitter['train'][i]['score'])) 
print(pipeline_as(dataset_twitter['train'][i]['texto']))

### TextBlob vs Pipeline (Twitter Roberta)

In [ ]:
def conserta_score(dado):
    resultado = dado['score'].split(',')[0]
    if resultado == 'Neutro':
        resultado = 'Neutral'
    elif resultado == 'Positivo':
        resultado = 'Positive'
    else:
        resultado = 'Negative'
    dado['score'] = resultado
    return dado

In [ ]:
dataset_twitter_teste = dataset_twitter['train'].map(conserta_score)

In [ ]:
print(dataset_twitter_teste[0])

In [ ]:
qtd_acerto = 0
for dado in dataset_twitter_teste:
    resultado = pipeline_as(dado['texto'])
    if resultado[0]['label'] == dado['score']:
        qtd_acerto += 1
print("Quantidade de matches de anailise de sentimento: " + str(qtd_acerto))
print("Procentagem de acerto: " + str((qtd_acerto/len(dataset_twitter_teste)) * 100))

### Validando Resultados

In [ ]:
import json
with open(DIR + 'resultados.json', 'w') as arquivo:
    json.dump(resultados, arquivo)

### Treinamento